In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import matplotlib
import time
import numpy as np
%matplotlib inline

import pandas as pd
pd.options.display.max_colwidth = 300
pd.options.display.max_columns = 100



In [2]:
#Model File Directory
WORK_DIR = '/var/spark/ml_files/'

#Label Keys
LABEL = "success"

MODEL_TYPE = "ML-TOD"

start_date = '2019-06-01'

end_date = '2019-06-30'

In [3]:
# from cassandra
from cassandra.cluster import Cluster
cassandra_endpoint = '10.81.12.121' #'10.62.1.118'
# cassandra_endpoint = '10.224.12.32'
cluster = Cluster([cassandra_endpoint])
# session = cluster.connect('subs')


In [5]:
FEATURES_CANDIDATES = [ 'issuer_country', 'day_of_month', 'site_id', 'merchant_number', 'transaction_hour',
                'payment_service_id', 'bin', 'payment_currency', 'bank_name', "card_category", "date_increment", 'decline_type']


USECOLS = FEATURES_CANDIDATES +  ['renew_att_num', 'cid' ,'payment_amount_usd', 'new_status', 'response_message', 'subscription_id', 'success', 'cid' ,'added_expiry_years', 'received_date', 'billing_country', 'transaction_date_in_string', 'cc_expiration_date']

In [6]:
TRAINING_DATA_PATH = WORK_DIR + 'tod_all_fields_2019_06.csv'
df =  pd.read_csv(TRAINING_DATA_PATH, usecols=USECOLS)
df['bin'] = df['bin'].fillna('').astype(str).str.replace('.0', '', regex=False)
df = df[~(df['new_status'] == 'Reversed')]
df.shape  #(1702331, 21)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1701778, 24)

In [7]:
EVAL_DATA_PATH = WORK_DIR + 'tod_all_fields_2019_07.csv'
df_eval =  pd.read_csv(EVAL_DATA_PATH, usecols=USECOLS)
df_eval['bin'] = df_eval['bin'].fillna('').astype(str).str.replace('.0', '', regex=False)
df_eval = df_eval[~(df_eval['new_status'] == 'Reversed')]
df_eval.shape

(1542736, 24)

In [8]:
TEST_DATA_PATH = WORK_DIR + 'tod_all_fields_2019_08.csv'
df_test =  pd.read_csv(TEST_DATA_PATH, usecols=USECOLS)
df_test['bin'] = df_test['bin'].fillna('').astype(str).str.replace('.0', '', regex=False)
df_test = df_test[~(df_test['new_status'] == 'Reversed')]
df_test.shape

(1709006, 24)

In [10]:
txn_hour_bin_dict = None
txn_hour_processor_dict = None
txn_hour_country_dict = None

TXN_HOUR_BIN_PATH = WORK_DIR + 'txn_hour_bin_2019_06_07.csv'
TXN_HOUR_PROCESSOR_PATH = WORK_DIR + 'txn_hour_processor_2019_06_07.csv'
TXN_HOUR_COUNTRY_PATH = WORK_DIR + 'txn_hour_country_2019_06_07.csv'
TXN_HOUR_SITE_PATH = WORK_DIR + 'txn_hour_site_2019_06_07.csv'
TXN_HOUR_BANK_NAME_PATH = WORK_DIR + 'txn_hour_bank_name_2019_06_07.csv'

TXN_HOUR_CURRENCY_PATH = WORK_DIR + 'txn_hour_currency_2019_06_07.csv'
TXN_HOUR_MID_PATH = WORK_DIR + 'txn_hour_mid_2019_06_07.csv'
TXN_HOUR_DATE_PATH = WORK_DIR + 'txn_hour_date_2019_06_07.csv'

TXN_HOUR_BIN_PROFILE_PATH = WORK_DIR + 'bin_profile_per_hour_2019_06_07.csv'
TXN_HOUR_PROCESSOR_PROFILE_PATH = WORK_DIR + 'processor_profile_per_hour_2019_06_07.csv'

if TXN_HOUR_BIN_PROFILE_PATH:
    bin_profile_per_hour = pd.read_csv(TXN_HOUR_BIN_PROFILE_PATH)
    bin_profile_per_hour.drop(columns=['Unnamed: 0'], inplace=True)
    bin_profile_per_hour['bin'] = bin_profile_per_hour['bin'].apply(str).str.replace('.0', '', regex=False)
    bin_profile_per_hour['transaction_hour'] = bin_profile_per_hour['transaction_hour'].apply(str).str.replace('.0', '', regex=False)
        
    bin_max_amt_per_hour_dict = bin_profile_per_hour.set_index(['bin', 'transaction_hour'])['Max_99'].T.to_dict()


if TXN_HOUR_PROCESSOR_PROFILE_PATH:
    processor_profile_per_hour = pd.read_csv(TXN_HOUR_PROCESSOR_PROFILE_PATH)
    processor_profile_per_hour.drop(columns=['Unnamed: 0'], inplace=True)
    processor_profile_per_hour['transaction_hour'] = processor_profile_per_hour['transaction_hour'].apply(str).str.replace('.0', '', regex=False)
        
    processor_max_amt_per_hour_dict = processor_profile_per_hour.set_index(['payment_service_id', 'transaction_hour'])['Max_99'].T.to_dict()    

if TXN_HOUR_BIN_PATH:
    txn_hour_bin = pd.read_csv(TXN_HOUR_BIN_PATH)
    txn_hour_bin.drop(columns=['Unnamed: 0'], inplace=True)
    txn_hour_bin['bin'] = txn_hour_bin['bin'].apply(str).str.replace('.0', '', regex=False)
    txn_hour_bin['transaction_hour'] = txn_hour_bin['transaction_hour'].apply(str).str.replace('.0', '', regex=False)
    txn_hour_bin_dict = txn_hour_bin[txn_hour_bin['num_of_subs'] >= 10].set_index(['bin', 'transaction_hour'])['success_rate'].T.to_dict()

    
# if  TXN_HOUR_PROCESSOR_PATH:
#     txn_hour_processor = pd.read_csv(TXN_HOUR_PROCESSOR_PATH)
#     txn_hour_processor.drop(columns=['Unnamed: 0'], inplace=True)
#     txn_hour_processor['processor_att_num'] = txn_hour_processor['processor_att_num'].apply(str).str.replace('.0', '', regex=False)
#     txn_hour_processor['transaction_hour'] = txn_hour_processor['transaction_hour'].apply(str).str.replace('.0', '', regex=False)
#     txn_hour_processor_dict = txn_hour_processor[txn_hour_processor['count'] >=10].set_index(['payment_service_id', 'processor_att_num', 'transaction_hour'])['success_rate'].T.to_dict()

if  TXN_HOUR_PROCESSOR_PATH:
    txn_hour_processor = pd.read_csv(TXN_HOUR_PROCESSOR_PATH)
    txn_hour_processor.drop(columns=['Unnamed: 0'], inplace=True)
    txn_hour_processor['transaction_hour'] = txn_hour_processor['transaction_hour'].apply(str).str.replace('.0', '', regex=False)
    txn_hour_processor_dict = txn_hour_processor[txn_hour_processor['num_of_subs'] >=10].set_index(['payment_service_id', 'transaction_hour'])['success_rate'].T.to_dict()

        
if  TXN_HOUR_COUNTRY_PATH:
    txn_hour_country = pd.read_csv(TXN_HOUR_COUNTRY_PATH)
    txn_hour_country.drop(columns=['Unnamed: 0'], inplace=True)
    txn_hour_country['transaction_hour'] = txn_hour_country['transaction_hour'].apply(str).str.replace('.0', '', regex=False)
    txn_hour_country_dict = txn_hour_country[txn_hour_country['num_of_subs'] >=10].set_index(['issuer_country', 'transaction_hour'])['success_rate'].T.to_dict()

    
if  TXN_HOUR_SITE_PATH:
    txn_hour_site = pd.read_csv(TXN_HOUR_SITE_PATH)
    txn_hour_site.drop(columns=['Unnamed: 0'], inplace=True)
    txn_hour_site['transaction_hour'] = txn_hour_site['transaction_hour'].apply(str).str.replace('.0', '', regex=False)
    txn_hour_site_dict = txn_hour_site[txn_hour_site['num_of_subs'] >=10].set_index(['site_id', 'transaction_hour'])['success_rate'].T.to_dict()
    
if  TXN_HOUR_BANK_NAME_PATH:
    txn_hour_bank_name = pd.read_csv(TXN_HOUR_BANK_NAME_PATH)
    txn_hour_bank_name.drop(columns=['Unnamed: 0'], inplace=True)
    txn_hour_bank_name['transaction_hour'] = txn_hour_bank_name['transaction_hour'].apply(str).str.replace('.0', '', regex=False)
    txn_hour_bank_name_dict = txn_hour_bank_name[txn_hour_bank_name['num_of_subs'] >=10].set_index(['bank_name', 'transaction_hour'])['success_rate'].T.to_dict()

if  TXN_HOUR_CURRENCY_PATH:
    txn_hour_currency = pd.read_csv(TXN_HOUR_CURRENCY_PATH)
    txn_hour_currency.drop(columns=['Unnamed: 0'], inplace=True)
    txn_hour_currency['transaction_hour'] = txn_hour_currency['transaction_hour'].apply(str).str.replace('.0', '', regex=False)
    txn_hour_currency_dict = txn_hour_currency[txn_hour_currency['num_of_subs'] >=10].set_index(['payment_currency', 'transaction_hour'])['success_rate'].T.to_dict()
    
if  TXN_HOUR_MID_PATH:
    txn_hour_mid = pd.read_csv(TXN_HOUR_MID_PATH)
    txn_hour_mid.drop(columns=['Unnamed: 0'], inplace=True)
    txn_hour_mid['transaction_hour'] = txn_hour_mid['transaction_hour'].apply(str).str.replace('.0', '', regex=False)
    txn_hour_mid_dict = txn_hour_mid[txn_hour_mid['num_of_subs'] >=10].set_index(['merchant_number', 'transaction_hour'])['success_rate'].T.to_dict()

if  TXN_HOUR_DATE_PATH:
    txn_hour_date = pd.read_csv(TXN_HOUR_DATE_PATH)
    txn_hour_date.drop(columns=['Unnamed: 0'], inplace=True)
    txn_hour_date['day_of_month'] = txn_hour_date['day_of_month'].apply(str).str.replace('.0', '', regex=False)
    txn_hour_date['transaction_hour'] = txn_hour_date['transaction_hour'].apply(str).str.replace('.0', '', regex=False)
    txn_hour_date_dict = txn_hour_date[txn_hour_date['num_of_subs'] >=10].set_index(['day_of_month', 'transaction_hour'])['success_rate'].T.to_dict()
    

In [11]:
original_size = len(df)
fail_size = df[LABEL].value_counts(normalize=True)[0.0]
success_size =  df[LABEL].value_counts(normalize=True)[1.0]

In [12]:
#import for training
import numpy as np
from sklearn import cross_validation
from sklearn import metrics
from sklearn.model_selection import cross_val_score

from sklearn import linear_model
from sklearn import tree
from sklearn import cross_validation
from sklearn import ensemble
from sklearn import linear_model
from sklearn import svm
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV
# from spark_sklearn import GridSearchCV
from sklearn.preprocessing import label_binarize
from sklearn.neighbors import KNeighborsClassifier

# from src.web.utils import PreProcessing
from src.web.preprocessing import PreProcessing
from src.web.encoder import EnhancedLeaveOneOutEncoder
from src.web.train_util import *

/opt/anaconda3/lib/python3.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [13]:
INPUT_FEATURES = {
            "billing_country": {
                "type": "string"
            },
            "bin": {
                "type": "string"
            },
            "bank_name": {
                "type": "string"
            },
            "card_brand": {
                "type": "string"
            },
            "card_category": {
                "type": "string"
            },
            "card_class": {
                "type": "string"
            },
            "card_usage": {
                "type": "string"
            },
            "day_of_month": {
                "type": "integer"
            },
            "funding_source": {
                "type": "string"
            },
            "issuer_country": {
                "type": "string"
            },
            "billing_country": {
                "type": "string"
            },
            "merchant_number": {
                "type": "string"
            },
            "payment_amount_usd": {
                "type": "number"
            },
            "payment_currency": {
                "type": "string"
            },
            "payment_method_id": {
                "type": "string"
            },
            "payment_service_id": {
                "type": "string"
            },
            "site_id": {
                "type": "string"
            },
            "transaction_date_in_string": {
                "type": "string"
            }
        }

In [29]:
# FEATURES_CAT = [ 'issuer_country',  'day_of_month', 'site_id', 'merchant_number', 
#                 'payment_service_id', 'bin', 'payment_currency', 'bank_name', 'transaction_hour']

FEATURES_CAT = [  'transaction_hour']

FEATURES_FLOAT = ['bin', 'renew_att_num']
FEATURES_NUM = []
FEATURES_NUM_CALCULATED = []

FEATURES_NUM_ENCODED = ['txn_hour_bin', 'txn_hour_processor', 'txn_hour_country', 'txn_hour_site', 'txn_hour_bank_name',
                        'txn_hour_currency', 'txn_hour_mid', 'txn_hour_date'] + FEATURES_NUM_BIN_PROFILE  #, 'payment_mid_bin'

FEATURES_GROUPED = []
FEATURES_CAT_ENCODED = [ 'week_of_month', 'day_of_week'] 
FEATURES_ENCODED = FEATURES_CAT_ENCODED + FEATURES_NUM_ENCODED

FEATURES = FEATURES_CAT + FEATURES_ENCODED

from sklearn.model_selection import GridSearchCV

from src.web.preprocessing import PreProcessing
from src.web.preprocessing import make_pipeline
from sklearn.preprocessing import Imputer

ADDITIONAL_FIELDS =  ['payment_amount_usd' ,'issuer_country',  'day_of_month', 'site_id', 'merchant_number', 
                'payment_service_id', 'bin', 'payment_currency', 'bank_name', 'transaction_date_in_string']
ADDITIONAL_FIELDS = [x for x in ADDITIONAL_FIELDS if x not in FEATURES_CAT]
FIELDS = FEATURES_CAT + FEATURES_NUM +  ADDITIONAL_FIELDS


# df_decline_type = pd.read_csv(WORK_DIR + 'Decline_Type.csv')

features_dict = {'LABEL': LABEL, 'FIELDS': FIELDS ,'FEATURES_CAT': FEATURES_CAT, 'FEATURES_NUM':FEATURES_NUM, 'FEATURES_ENCODED':FEATURES_ENCODED, 'FEATURES_NUM_ENCODED':FEATURES_NUM_ENCODED, 'FEATURES_NUM_CALCULATED':FEATURES_NUM_CALCULATED, 'FEATURES_FLOAT': FEATURES_FLOAT}
features_dict_key = 'preprocessing__features_dict'
features_dict['df_bin_profile'] = None #bin_profile
features_dict['FEATURES_GROUPED'] = FEATURES_GROUPED


features_dict['txn_hour_group_dict'] = {"txn_hour_bin_dict": txn_hour_bin_dict, "txn_hour_processor_dict": txn_hour_processor_dict,\
                               "txn_hour_country_dict": txn_hour_country_dict, 'txn_hour_site_dict': txn_hour_site_dict, \
                               "txn_hour_currency_dict": txn_hour_currency_dict, 'txn_hour_mid_dict': txn_hour_mid_dict, \
                               "txn_hour_bank_name_dict": txn_hour_bank_name_dict, 'txn_hour_date_dict': txn_hour_date_dict}

In [19]:
exclude_decline_types = ['invalid_account', 'invalid_cc', 'correct_cc_retry', 'expired_card']
df_train = df[df['date_increment'].isna()]
df_train = df_train[~(df_train['decline_type'].isin(exclude_decline_types))]

df_eval = df_eval[df_eval['date_increment'].isna()]
df_eval = df_eval[~(df_eval['decline_type'].isin(exclude_decline_types))]

df_test = df_test[df_test['date_increment'].isna()]
df_test = df_test[~(df_test['decline_type'].isin(exclude_decline_types))]

#Exclude some data
df_train = df_train[~(df_train['payment_service_id'] == 'paypalExpress')]
df_eval = df_eval[~(df_eval['payment_service_id'] == 'paypalExpress')]
df_test = df_test[~(df_test['payment_service_id'] == 'paypalExpress')]

df_train = df_train[~(df_train['bin'] == 'nan')]
df_eval = df_eval[~(df_eval['bin'] == 'nan')]
df_test = df_test[~(df_test['bin'] == 'nan')]

df_train = df_train[~(df_train['cc_expiration_date'] == 'nan')]
df_eval = df_eval[~(df_eval['cc_expiration_date'] == 'nan')]
df_test = df_test[~(df_test['cc_expiration_date'] == 'nan')]

In [ ]:
""" Train the model"""

from catboost import CatBoostClassifier, Pool
import src.web.preprocessing
from src.web.preprocessing import PreProcessing
from src.web.train_util import *
from importlib import import_module
import sys

classifier = CatBoostClassifier

cat_features_len = len(FEATURES_CAT) + len(FEATURES_CAT_ENCODED) + len (FEATURES_GROUPED)
input_data = df_train #retry_success[~retry_success["subscription_id"].isin(duplicate_subs)]
scale_pos_weight = (input_data[LABEL].value_counts(normalize=True)[0.0] / input_data[LABEL].value_counts(normalize=True)[1.0] ) + 0.5

features_dict['use_cat_encoder'] = False
_preProcessor = PreProcessing(None).fit(input_data, input_data['success'], features_dict=features_dict)            
_x_eval = _preProcessor.transform(df_eval)
_y_eval = df_eval["success"]

alg_name = 'catboostclassifier'

tuned_parameters = {}

best_parameters = {
              'depth': 7,
              'iterations': 1000,
              'random_seed': 8,
              'scale_pos_weight': scale_pos_weight,
              'subsample': 0.3,
              'bagging_temperature': 3.0,
              'rsm': 0.35,
              'eval_metric': 'AUC',
              'early_stopping_rounds': 200,
              'l2_leaf_reg': 10.9,
              'random_strength': 3.0
              }

#               'border_count': 260,
    
model_file = ''
model_id = 'ML-TOD-1'
version = get_latest_version(model_id, MODEL_TYPE) + 1
model_name = model_id + '.' + str(version)

# features_dict['eval_metric'] = 'auc'
# features_dict['early_stopping_rounds'] = None #1600
# features_dict['eval_set'] = None #Pool(_x_eval, _y_eval) 

cat_features = list(range(0,cat_features_len))

fit_params = {
    f"{alg_name}__verbose": True,
    f"{alg_name}__cat_features": cat_features,
    f"{alg_name}__plot": True,
    f"{alg_name}__eval_set": Pool(_x_eval, _y_eval, cat_features)
}


features_dict['fit_params'] = fit_params


clf, result_d = build_and_train(input_data, classifier, tuned_parameters, alg_name, model_file, best_param=best_parameters, features_dict=features_dict, test_data=df_test)
model_file, model_file_name = write_model(clf, model_name)
print("model_file is generated: ", model_file)
print("result_dict: ", result_d)


df_bin_profile is NONE
self.features_encoded: ['week_of_month', 'day_of_week']


In [32]:
"""Upload model to Nexus repo and insert the model info into Cassandra table"""
import json

# start_date = '2018-08-01'
start_date = '2019-01-01'
# start_date = '2019-03-01'

# end_date = '2019-04-30'
end_date = '2019-07-31'

try:
    repo_path = upload_artifact(model_file_name)
    preprocess_repo_path = handle_preprocessing_file(model_id, version)
    size_desc = str(", original size: %s (fail: %s, success: %s), balanced_size: %s" % (original_size, fail_size, success_size, original_size))
    desc = '{}_{}_for_calendar retry model,  eval_metric= auc, with no date_increment, no payment amount and bin profile). {}'.format(start_date, end_date, size_desc)
    hyper_params = result_d.pop('hyper_params', None)
    extended_att = {"preprocess_repo_path": preprocess_repo_path, "input_features": INPUT_FEATURES}
    insert_model_info(model_id, version, repo_path, desc=desc, model_type=MODEL_TYPE,eval_metrics=json.dumps(result_d), 
                      hyper_parameter=json.dumps(hyper_params), extended_att=json.dumps(extended_att), features_dict=features_dict, algorithm='CatBoostClassifier')
    
except Exception as ex:
    print(str(ex))
    if not hyper_params:
        result_d['hyper_params'] = hyper_params

<Response [201]>
http://nexus-master.digitalriverws.net/nexus/repository/foundationreleases/com/digitalriver/prediction-service/ML-BR/1.190/ML-BR-1.190.pkl
[com.digitalriver.prediction-service:ML-BR:1.190::pkl]


The following files were uploaded to repository foundationreleases


<Response [201]>
http://nexus-master.digitalriverws.net/nexus/repository/foundationreleases/com/digitalriver/prediction-service/ML-BR/1_190_preprocessing/ML-BR-1_190_preprocessing.py
[com.digitalriver.prediction-service:ML-BR:1_190_preprocessing::py]
Model ML-BR-1 version 190 is inserted into model repo


The following files were uploaded to repository foundationreleases
